In [1]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='./../.env')
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'),temperature=0.7)
dspy.configure(lm=lm)

/Users/meguru_kiss/anaconda3/envs/textsum/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
test_file_path='~/Downloads/test 2.csv'
test_dataset=pd.read_csv(test_file_path)
test_dataset.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [4]:
test_dataset.iloc[0]['article']

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [5]:
test_dataset.iloc[0]['highlights']

'Experts question if  packed out planes are putting passengers at risk .\nU.S consumer advisory group says minimum space must be stipulated .\nSafety tests conducted on planes with more leg room than airlines offer .'

In [16]:
from dspy_utils import PersonalizedSummary
summarize = dspy.ChainOfThought(PersonalizedSummary)
response = summarize(document=test_dataset.iloc[0]['article'],length=50, style="formal", context="")

In [17]:
print(response.summary)

Experts warn that decreasing airplane seat sizes compromise passenger health and safety, as regulations prioritize space for animals over humans. The FAA's tests indicate that many airlines now offer insufficient seat pitch, raising serious concerns about overcrowding on flights.


In [10]:
dspy.inspect_history(n=1)





[2024-12-06T20:58:21.012282]

System message:

Your input fields are:
1. `document` (str): The document to summarize
2. `length` (int): The maximum length of the summary
3. `style` (str): The style of the summary, like 'formal', 'casual'
4. `context` (str): May contain relevant facts

Your output fields are:
1. `reasoning` (str)
2. `summary` (str): The summary of the document

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## document ## ]]
{document}

[[ ## length ## ]]
{length}

[[ ## style ## ]]
{style}

[[ ## context ## ]]
{context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `document`, `length`, `style`, `context`, produce the fields `summary`.


User message:

[[ ## document ## ]]
Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skie

In [18]:
from dspy_utils import Assess
summary=response.summary
assess = dspy.ChainOfThought(Assess)
response = assess(original_document=test_dataset.iloc[0]['article'],summary_text=summary, length=50, style="formal", context="")

In [19]:
response

Prediction(
    reasoning='The summary effectively captures the main points of the original document, addressing the concerns about shrinking airplane seat sizes and their implications for passenger health and safety. It maintains a formal tone throughout, aligning with the requested style. The length of the summary is concise and fits within the specified limit of 50 words. Overall, it provides a clear and informative overview of the key issues presented in the original text.',
    score=90
)

In [20]:
response = assess(original_document=test_dataset.iloc[0]['article'],summary_text=test_dataset.iloc[0]['highlights'], length=50, style="formal", context="")

In [21]:
response

Prediction(
    reasoning="The summary captures the essence of the original document by addressing the concerns raised by experts regarding the shrinking space on planes and the potential risks to passenger safety. It mentions the U.S. consumer advisory group's call for minimum space requirements, which aligns well with the formal style requested. However, the summary could be slightly more detailed to convey the full scope of the issues discussed, particularly regarding the specific seat pitches mentioned. Overall, it meets the length requirement and maintains a formal tone.",
    score=80
)

In [22]:
dspy.inspect_history(n=1)





[2024-12-06T21:04:54.631999]

System message:

Your input fields are:
1. `original_document` (str): The original document to summarize
2. `length` (int): The maximum length of the summary
3. `style` (str): The style of the summary, either 'formal' or 'casual'
4. `summary_text` (str): The summary text, evaluated based on whether the length, and style personalization preferences are met

Your output fields are:
1. `reasoning` (str)
2. `score` (int): The score of the summary, between 0 and 100

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## original_document ## ]]
{original_document}

[[ ## length ## ]]
{length}

[[ ## style ## ]]
{style}

[[ ## summary_text ## ]]
{summary_text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## score ## ]]
{score}        # note: the value you produce must be a single int value

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `original_document`, `length